In [43]:
# делаем import необходимых библиотек
import os
import mlflow
import psycopg
import pandas as pd

In [44]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20241121_573777dca2",
    "user": "mle_20241121_573777dca2",
    "password": "4ebaa3f6f3b04c68aeaeb0984371b617",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [45]:
df

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,17,8191-XWSZG,2015-10-01,NaT,One year,No,Mailed check,20.65,1022.95,None,...,None,None,None,None,Female,0,No,No,No,0
1,59,3957-SQXML,2017-04-01,NaT,Two year,No,Credit card (automatic),24.95,894.30,None,...,None,None,None,None,Female,0,Yes,Yes,Yes,0
2,148,6837-BJYDQ,2019-11-01,NaT,One year,No,Mailed check,19.60,61.35,None,...,None,None,None,None,Male,0,No,No,No,0
3,482,0486-LGCCH,2019-03-01,NaT,Two year,No,Mailed check,19.65,225.75,None,...,None,None,None,None,Male,0,Yes,Yes,No,0
4,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,13002,1035-IPQPU,2014-11-01,NaT,Month-to-month,Yes,Electronic check,103.50,6479.40,Fiber optic,...,Yes,No,Yes,Yes,Female,1,Yes,No,Yes,0
7039,13004,7398-LXGYX,2016-06-01,NaT,Month-to-month,Yes,Credit card (automatic),84.80,3626.35,Fiber optic,...,Yes,No,No,No,Male,0,Yes,No,Yes,0
7040,13006,2823-LKABH,2018-08-01,NaT,Month-to-month,Yes,Bank transfer (automatic),95.05,1679.40,Fiber optic,...,Yes,Yes,No,Yes,Female,0,No,No,Yes,0
7041,13008,8775-CEBBJ,2019-02-01,2019-11-01,Month-to-month,Yes,Bank transfer (automatic),44.20,403.35,DSL,...,No,No,No,No,Female,0,No,No,No,1


In [46]:
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(",".join(df.columns))

In [47]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

In [48]:
stats

{'type_Month-to-month': 3875,
 'type_Two year': 1695,
 'type_One year': 1473,
 'paperless_billing_Yes': 4171,
 'paperless_billing_No': 2872,
 'internet_service_Fiber optic': 3096,
 'internet_service_DSL': 2421,
 'online_security_No': 3498,
 'online_security_Yes': 2019,
 'online_backup_No': 3088,
 'online_backup_Yes': 2429,
 'device_protection_No': 3095,
 'device_protection_Yes': 2422,
 'tech_support_No': 3473,
 'tech_support_Yes': 2044,
 'streaming_tv_No': 2810,
 'streaming_tv_Yes': 2707,
 'streaming_movies_No': 2785,
 'streaming_movies_Yes': 2732,
 'gender_Male': 3555,
 'gender_Female': 3488,
 'senior_citizen_0': 5901,
 'senior_citizen_1': 1142,
 'partner_No': 3641,
 'partner_Yes': 3402,
 'dependents_No': 4933,
 'dependents_Yes': 2110,
 'multiple_lines_No': 3390,
 'multiple_lines_Yes': 2971,
 'target_0': 5174,
 'target_1': 1869,
 'data_length': 7043,
 'monthly_charges_min': 18.25,
 'monthly_charges_max': 118.75,
 'monthly_charges_mean': 64.76169246059918,
 'monthly_charges_median': 70

In [49]:
df.to_csv("users_churn.csv", index=False)

In [50]:
# задаём название эксперимента и имя запуска для логирования в MLflow
EXPERIMENT_NAME = "churn_ivan_p2"
RUN_NAME = "data_check"

# создаём новый эксперимент в MLflow с указанным названием
# если эксперимент с таким именем уже существует,
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id

    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # объявлять переменную stats не надо,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)

    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', artifact_path='dataframe')
    mlflow.log_artifact('users_churn.csv', artifact_path='dataframe')

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)

# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того,
# что эксперимент был завершён успешно
assert run.info.status == 'FINISHED'

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
# os.remove('columns.txt')
# os.remove('users_churn.csv')


RestException: RESOURCE_ALREADY_EXISTS: Experiment(name=churn_ivan_p2) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "experiments_name_key"
DETAIL:  Key (name)=(churn_ivan_p2) already exists.

[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (%(name)s, %(artifact_location)s, %(lifecycle_stage)s, %(creation_time)s, %(last_update_time)s) RETURNING experiments.experiment_id]
[parameters: {'name': 'churn_ivan_p2', 'artifact_location': '', 'lifecycle_stage': 'active', 'creation_time': 1753999254260, 'last_update_time': 1753999254260}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, log_loss
)
from mlflow.models import infer_signature

In [52]:

# Prepare the features
X = df.drop(['id', 'customer_id', 'begin_date', 'end_date', 'target'], axis=1)
y = df['target']

# Convert categorical variables to dummy variables
X = pd.get_dummies(X, columns=['type', 'paperless_billing', 'payment_method', 'internet_service', 
                               'online_security', 'online_backup', 'device_protection', 'tech_support', 
                               'streaming_tv', 'streaming_movies', 'gender', 'partner', 'dependents', 
                               'multiple_lines'], drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Create and train the model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42)

In [53]:
# Make predictions
prediction = model.predict(X_test_scaled)
probas = model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
metrics = {}

cm = confusion_matrix(y_test, prediction)
tn, fp, fn, tp = cm.ravel()
err1 = fp / (fp + tn)  # False Positive Rate
err2 = fn / (fn + tp)  # False Negative Rate
auc = roc_auc_score(y_test, probas)
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, probas)

metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss
metrics["accuracy"] = accuracy_score(y_test, prediction)

In [54]:
# Log the model and metrics using MLflow
EXPERIMENT_NAME = "churn_model_test"
RUN_NAME = "logistic_regression_extended_metrics"

mlflow.set_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME) as run:
    mlflow.log_params(model.get_params())
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_param("features", list(X.columns))
    run_id = run.info.run_id

print(f"Model logged to MLflow with run ID: {run_id}")
print("Metrics:", metrics)

2025/07/31 22:01:00 INFO mlflow.tracking.fluent: Experiment with name 'churn_model_test' does not exist. Creating a new experiment.


Model logged to MLflow with run ID: b32e45f15a9e442d88b46a9904080ce7
Metrics: {'err1': 0.09803921568627451, 'err2': 0.4910025706940874, 'auc': 0.8336559302384192, 'precision': 0.6644295302013423, 'recall': 0.5089974293059126, 'f1': 0.5764192139737991, 'logloss': 0.4353242325682949, 'accuracy': 0.7934705464868701}


In [57]:
import os
import mlflow
from mlflow.models import infer_signature

# MLflow and S3 configuration
EXPERIMENT_NAME = "churn_ivan_prediction_exp3"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_ivan_p3"

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"

# Set the tracking URI and configure S3 storage
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(EXPERIMENT_NAME)

# Ensure the experiment is created with the correct S3 artifact location
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    artifact_location = "s3://s3-student-mle-20241121-573777dca2"
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME, artifact_location=artifact_location)
else:
    experiment_id = experiment.experiment_id

# Model requirements and metadata
pip_requirements = os.path.abspath('../requirements.txt')
signature = infer_signature(X_test_scaled, prediction)
input_example = X_test_scaled[:10]
metadata = {'model_type': 'monthly'}

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    # Log metrics
    mlflow.log_metrics(metrics)
    
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="models",
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        registered_model_name=REGISTRY_MODEL_NAME,
        await_registration_for=60
    )

print(f"Model registered with run ID: {run_id}")
print(f"Model URI: {model_info.model_uri}")

# Retrieve the latest version of the registered model
client = mlflow.tracking.MlflowClient()
registered_model = client.get_registered_model(REGISTRY_MODEL_NAME)
latest_version = registered_model.latest_versions[0].version

print(f"Model version: {latest_version}")

2025/07/31 22:01:41 INFO mlflow.tracking.fluent: Experiment with name 'churn_ivan_prediction_exp3' does not exist. Creating a new experiment.


Successfully registered model 'churn_model_ivan_p3'.
2025/07/31 22:01:41 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_ivan_p3, version 1


Model registered with run ID: d9b792a9a2364122a30cb214cc471e38
Model URI: runs:/d9b792a9a2364122a30cb214cc471e38/models
Model version: 1


Created version '1' of model 'churn_model_ivan_p3'.


In [40]:
print(os.environ.get("MLFLOW_S3_ENDPOINT_URL"))
print(os.environ.get("AWS_ACCESS_KEY_ID"))
print(os.environ.get("AWS_SECRET_ACCESS_KEY"))

https://storage.yandexcloud.net
YCAJE3Nlz8iDILW5VTYM1ihQB
YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44


In [42]:
import boto3
import os

# S3 configuration
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
endpoint_url = "https://storage.yandexcloud.net"
bucket_name = "s3-student-mle-20241121-573777dca2"
folder_path = "3/f0b7cbf863894ab9a136758f13650fdd/artifacts/models/"

# Create an S3 client
s3 = boto3.client('s3', endpoint_url=endpoint_url)

try:
    # List objects in the specific folder
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)

    if 'Contents' in response:
        print(f"Objects in folder '{folder_path}':")
        for obj in response['Contents']:
            print(f" - {obj['Key']}")
    else:
        print(f"The folder '{folder_path}' is empty or does not exist.")

    # Check for specific files
    expected_files = ['MLmodel', 'conda.yaml', 'python_env.yaml', 'requirements.txt', 'input_example.json']
    found_files = [obj['Key'].split('/')[-1] for obj in response.get('Contents', [])]
    
    print("\nChecking for expected files:")
    for file in expected_files:
        if file in found_files:
            print(f" - {file}: Found")
        else:
            print(f" - {file}: Not found")

except Exception as e:
    print(f"An error occurred: {str(e)}")

The folder '3/f0b7cbf863894ab9a136758f13650fdd/artifacts/models/' is empty or does not exist.

Checking for expected files:
 - MLmodel: Not found
 - conda.yaml: Not found
 - python_env.yaml: Not found
 - requirements.txt: Not found
 - input_example.json: Not found


In [58]:
model_info.model_uri

'runs:/d9b792a9a2364122a30cb214cc471e38/models'

In [59]:
# Load the model
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

# Make predictions
model_predictions = loaded_model.predict(X_test).astype(int)

# Verify the predictions are integers
assert model_predictions.dtype == int

# Print the first 10 predictions
print(model_predictions[:10])



MlflowException: dtype of input object does not match expected dtype float64

In [33]:
model_dependencies = mlflow.pyfunc.get_model_dependencies(model_uri=model_info.model_uri)

2025/07/31 21:36:43 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmpas4t00fm/models/requirements.txt'.


In [35]:
!pip install -r {model_dependencies}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 20.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 KB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 46.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.3 MB/s eta 0:00:0000:0100:01
  Using cached catboost-1.2.2-cp310-cp310-manylinux2014_x86_64.whl (98.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 30.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 KB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.8 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 32.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [36]:
# Load the model
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

# Make predictions
model_predictions = loaded_model.predict(X_test).astype(int)

# Verify the predictions are integers
assert model_predictions.dtype == int

# Print the first 10 predictions
print(model_predictions[:10])

MlflowException: dtype of input object does not match expected dtype float64